In [1]:
import os
import numpy as np
import mne  # Ensure you have MNE installed
from sklearn.metrics import accuracy_score, classification_report
from aeon.classification.deep_learning import ResNetClassifier
from sklearn.model_selection import train_test_split

def parse_seizure_times_from_file(summary_file_path):
    seizure_times = []
    with open(summary_file_path, 'r') as file:
        lines = file.readlines()
        start_time = None
        for line in lines:
            if 'Seizure Start Time:' in line:
                start_time = int(line.split(':')[1].strip().split()[0])  # Extract number only
            elif 'Seizure End Time:' in line and start_time is not None:
                end_time = int(line.split(':')[1].strip().split()[0])  # Extract number only
                seizure_times.append((start_time, end_time))
                start_time = None  # Reset start_time after capturing a seizure
    return seizure_times

def extract_segments(edf_file_path, seizure_times, segment_duration=2, max_segments=100):
    raw = mne.io.read_raw_edf(edf_file_path, preload=True)
    data = raw.get_data()  # Shape: (n_channels, n_timepoints)
    sfreq = raw.info['sfreq']  # Sampling frequency
    segment_samples = int(segment_duration * sfreq)  # Convert duration to samples
    n_channels = data.shape[0]
    
    X = []
    y = []

    for start, _ in seizure_times:
        preictal_start = max(start - 1800, 0)  # 30 minutes before the seizure
        preictal_end = start
        
        for i in range(preictal_start, preictal_end, segment_samples):
            if len(X) >= max_segments:
                break
            if i + segment_samples <= preictal_end:
                segment = data[:, i:i + segment_samples]
                X.append(segment)
                y.append(1)

    for start, _ in seizure_times:
        interictal_start = max(start - 14400, 0)  # 4 hours before the seizure
        
        for i in range(interictal_start, start - 1800, segment_samples):
            if len(X) >= max_segments:
                break
            if i + segment_samples <= start - 1800:
                segment = data[:, i:i + segment_samples]
                X.append(segment)
                y.append(0)

    X = np.array(X)
    y = np.array(y)
    
    return X, y

def load_data(folder_path, seizure_times, max_segments_per_file=100):
    X = []
    y = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.edf'):
            edf_file_path = os.path.join(folder_path, filename)
            X_segments, y_segments = extract_segments(edf_file_path, seizure_times, max_segments=max_segments_per_file)
            X.append(X_segments)
            y.append(y_segments)

    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    
    return X, y

# Define paths
summary_file_path = "D:/chb-mit-scalp-eeg-database-1.0.0/chb02/chb02-summary.txt"
folder_path = "D:/chb-mit-scalp-eeg-database-1.0.0/chb02"

# Load seizure times from the summary file
seizure_times = parse_seizure_times_from_file(summary_file_path)

# Load data
X, y = load_data(folder_path, seizure_times)

# Initialize the ResNetClassifier
classifier = ResNetClassifier(
    n_epochs=150,
    batch_size=64,
    verbose=True,
    n_residual_blocks=3,
)

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the ResNet classifier
classifier.fit(X_train, y_train)

# Predict on the test set
y_test_pred = classifier.predict(X_test)

# Evaluate on the test set
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))


C:\Users\HP\anaconda3\Lib\site-packages\aeon\base\__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


Extracting EDF parameters from D:\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\HP\AppData\Local\Temp\ipykernel_45776\1869720676.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file_path, preload=True)


Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Extracting EDF parameters from D:\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\HP\AppData\Local\Temp\ipykernel_45776\1869720676.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file_path, preload=True)


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 512, 23)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 512, 64)           │          11,840 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 512, 64)           │             256 │ conv1d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 512, 64)           │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 512, 64)           │          20,544 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 512, 64)           │             256 │ conv1d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 512, 64)           │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_2 (Conv1D)             │ (None, 512, 64)           │          12,352 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_3 (Conv1D)             │ (None, 512, 64)           │           1,536 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 512, 64)           │             256 │ conv1d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 512, 64)           │             256 │ conv1d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 512, 64)           │               0 │ batch_normalization_2[0][… │
│                               │                           │                 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 512, 64)           │               0 │ add[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_4 (Conv1D)             │ (None, 512, 128)          │          65,664 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_4         │ (None, 512, 128)          │             51

 Total params: 536,002 (2.04 MB)

 Trainable params: 533,442 (2.03 MB)

 Non-trainable params: 2,560 (10.00 KB)

Epoch 1/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.4653 - loss: 1.2183 - learning_rate: 0.0100
Epoch 2/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.4322 - loss: 0.7421 - learning_rate: 0.0100
Epoch 3/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5499 - loss: 0.6935 - learning_rate: 0.0100
Epoch 4/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5551 - loss: 0.6752 - learning_rate: 0.0100
Epoch 5/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5960 - loss: 0.6650 - learning_rate: 0.0100
Epoch 6/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.6066 - loss: 0.6626 - learning_rate: 0.0100
Epoch 7/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.6101 - loss: 0.6590 - learning_rate: 0.0100
Epoch 8/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5990 - loss: 0.6496 - learning_rate: 0.0100
Epoch 9/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.6249 - loss: 0.6394 - learning_rate: 0.0100
Epoch 10/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/